# Intro to gragh convolutional surrogate models
Eamon Whalen

In [1]:
import sys
import numpy as np
import pandas as pd
import altair as alt

sys.path.append('./models')
from feastnetSurrogateModel import FeaStNet

sys.path.append('./readers')
from loadGhGraphs import loadGhGraphs

sys.path.append('./visualization')
from altTrussViz import plotTruss, interactiveErrorPlot

sys.path.append('./util')
from gcnSurrogateUtil import *

## 1. Load simulation data

In [2]:
doeFile = "/home/ewhalen/projects/data/trusses/2D_Truss_v1.3/design_7_N_1000.csv"
allGraphsUnfiltered = loadGhGraphs(doeFile, NUM_DV=5)

maxes = [max(np.abs(graph.y.numpy().flatten())) for graph in allGraphsUnfiltered]
source = pd.DataFrame(maxes, columns=['maxes'])
source.describe()

,maxes
count,1000.000000
mean,0.199516
std,2.524026
min,0.006570
25%,0.015425
50%,0.024334
75%,0.046981
max,67.325867


In [3]:
plotTruss(allGraphsUnfiltered[0], showDeformed=True, defScale=100)

alt.LayerChart(...)

## 2. Filter and partition

In [4]:
allGraphs = filterbyDisp(allGraphsUnfiltered, 0.9)
trainData, valData, testData = partitionGraphList(allGraphs)

maxes = [max(np.abs(graph.y.numpy().flatten())) for graph in allGraphs]
source = pd.DataFrame(maxes, columns=['maxes'])
source.describe()

,maxes
count,900.000000
mean,0.028952
std,0.019071
min,0.006570
25%,0.014951
50%,0.021719
75%,0.038367
max,0.097861


## 3. Train a GCN

In [5]:
gcn = FeaStNet()
history = gcn.trainModel(trainData, valData, 
                     epochs=100, 
                     batch_size=256, 
                     flatten=True, 
                     logTrans=False, 
                     ssTrans=True, 
                     saveDir='./results/gcn01/')

plotHistory(history)

epoch: 0   trainLoss: 1.0989e+00   valLoss:1.0126e+00  time: 5.83e+00
epoch: 1   trainLoss: 1.0456e+00   valLoss:1.0258e+00  time: 1.00e+00
epoch: 2   trainLoss: 9.5005e-01   valLoss:1.0426e+00  time: 9.74e-01
epoch: 3   trainLoss: 9.1063e-01   valLoss:1.0338e+00  time: 1.07e+00
epoch: 4   trainLoss: 8.5706e-01   valLoss:1.0135e+00  time: 1.01e+00
epoch: 5   trainLoss: 8.7558e-01   valLoss:1.0095e+00  time: 9.80e-01
epoch: 6   trainLoss: 8.1543e-01   valLoss:9.4506e-01  time: 1.07e+00
epoch: 7   trainLoss: 7.9429e-01   valLoss:8.3523e-01  time: 9.75e-01
epoch: 8   trainLoss: 7.4337e-01   valLoss:7.5464e-01  time: 9.86e-01
epoch: 9   trainLoss: 7.2737e-01   valLoss:7.1319e-01  time: 9.96e-01
epoch: 10   trainLoss: 6.8807e-01   valLoss:6.9046e-01  time: 1.12e+00
epoch: 11   trainLoss: 6.7639e-01   valLoss:6.7010e-01  time: 9.77e-01
epoch: 12   trainLoss: 6.4848e-01   valLoss:6.4475e-01  time: 9.77e-01
epoch: 13   trainLoss: 6.0920e-01   valLoss:6.1865e-01  time: 9.81e-01
epoch: 14   trai

alt.Chart(...)

## 4. Test the GCN

In [6]:
trainRes = gcn.testModel(trainData)
testRes = gcn.testModel(testData)
pd.DataFrame([trainRes, testRes], index=['train', 'test'])

,mse,mae,mre,peakR2,maxAggR2,meanAggR2,minAggR2
train,0.000015,0.002565,0.115431,0.881693,0.950303,0.501409,-4.337326
test,0.000023,0.002942,0.131308,0.785645,0.906220,0.366116,-5.358913


## 5. Visualize some predictions

In [7]:
i = 9
pred = gcn.predict([testData[i]])[0]
plotTruss(testData[i], showDeformed=True, defScale=100, prediction=pred)

alt.LayerChart(...)

In [9]:
# interactive scatter plot
alt.data_transformers.enable('json')
allPreds = gcn.predict(testData)
interactiveErrorPlot(testData, allPreds)

alt.HConcatChart(...)

In [21]:
x, y = np.meshgrid(range(-5, 5), range(-5, 5))
z = x ** 2 + y ** 2

df = pd.DataFrame({'x': x.ravel(),
                     'y': y.ravel(),
                     'z': z.ravel()})

contour = alt.Chart(df).mark_rect().encode(
    x='x:O',
    y='y:O',
    color='z:Q'
).properties(width=100, height=100)


i = 171
pred = gcn.predict([testData[i]])[0]
plotTruss(testData[i], showDeformed=False)+contour

alt.LayerChart(...)